export a unique set of data synthesized

In [18]:
    import requests_cache, platformdirs, requests_cache, pandas, urllib
    from toolz.curried import *
    from pandas import Series, DataFrame, Index
    from pathlib import Path
    cache = platformdirs.user_cache_path("a11yhood") / "thingiverse"
    cache = Path("data/thingiverse")
    CachedSession = partial(requests_cache.CachedSession, backend="filesystem", serializer="json")
    search_session = CachedSession(cache / "search_responses")
    thing_session = CachedSession(cache / "thing_responses")

In [19]:
    def get_thing_details_tidy(df, index):
        df = df[df.columns.difference(index.columns)]
        df = df.join(
            df.pop("zip_data").apply(Series).stack().explode().apply(operator.itemgetter("url")).groupby(level=0).agg(list).rename("files")
        )
        # the default image is in the files
        df.pop("default_image");
        df.pop("edu_details_parts");
        df.pop("education");
        df.pop("description_html");
        # TODO there is information in details_parts than can be extracted for search.
        details_parts = df.pop("details_parts").explode().apply(Series).set_index("type", append=True)
        # detail_parts is segmented string info we cram into  a single block
        details_parts = details_parts.join(
            details_parts.data.explode().dropna().apply(Series).fillna("").astype(str).apply("\n".join, axis=1).rename("parts")
        )
        details_parts = details_parts.reset_index(1, drop=True)
        df = df.join(
            (("<h2>" + details_parts["name"] + "</h2>") + "\n" + details_parts["parts"]).dropna().groupby(level=0).agg("\n".join).rename("details_parts")
        )
        df["ancestors"] = df.ancestors.explode().dropna().apply(Series).public_url.groupby(level=0).agg(list)
        return df

In [20]:
    things = (
        df := Series(search_session.cache.responses)
        .apply(operator.methodcaller("json"))
        .apply(operator.itemgetter("hits"))
        .explode()
        .apply(Series)
        .set_index("id")
    ).join(
        Series(thing_session.cache.responses)
        .apply(operator.methodcaller("json"))
        .apply(Series).pipe(get_thing_details_tidy, df)
        .set_index("id")
    )
    things.tags = (
        things.tags
        .explode()
        .dropna()
        .apply(operator.itemgetter("name"))
        .groupby(pandas.Grouper(level=0))
        .agg(list)
    )

## ravelry

In [21]:
    import requests_cache, platformdirs
    cache = platformdirs.user_cache_path("a11yhood") / "ravelry"
    cache = Path("data/ravelry")
    search_cache = CachedSession(cache / "search_responses")
    patterns_cache = CachedSession(cache / "patterns_responses")

In [22]:
    patterns = (
        df := Series(search_cache.cache.responses)
        .apply(operator.methodcaller("json"))
        .apply(operator.itemgetter("patterns"))
        .explode()
        .apply(Series)
        .set_index("id")
    ).join(
        (
            g:= Series(patterns_cache.cache.responses)
            .apply(operator.methodcaller("json"))
            .apply(operator.itemgetter("patterns"))
            .apply(dict.values)
            .explode()
            .apply(Series)
            .set_index("id")
        )[g.columns.difference(df.columns)]
    )

In [23]:
    patterns = patterns.join(pandas.concat([
        patterns.pattern_categories.explode().apply(Series).name,
        patterns.pattern_attributes.explode().apply(Series).permalink.str.replace("-", " ")
    ], axis=0).groupby(level=0).agg(list).rename("tags"))

## github

In [24]:
    cache = platformdirs.user_cache_path("a11yhood") / "github"
    search_cache = cache / "search_responses.pkl"

In [25]:
    with __import__("shelve").open(search_cache) as db:
        (
            repos := Series(db)
            .apply(operator.itemgetter("data"))
            .apply(operator.itemgetter("search"))
            .apply(operator.itemgetter("edges"))
            .explode()
            .apply(operator.itemgetter("node"))
            .apply(Series).set_index("id")
        )

    repos["tags"] = (
        repos.repositoryTopics
        .apply(operator.itemgetter("edges"))
        .explode()
        .apply(operator.itemgetter("node"))
        .apply(operator.itemgetter("topic"))
        .apply(operator.itemgetter("name"))
        .str.replace("-", " ")
        .groupby(pandas.Grouper(level=0)).agg(list)
    )

    repos["issues"] = repos["issues"].apply(get("totalCount"))
    repos["pullRequests"] = repos["pullRequests"].apply(get("totalCount"))
    repos = repos.drop(columns="licenseInfo").join(repos["licenseInfo"].dropna().apply(get("name")))

## text analysis

`all` is a synthesis of each of the services with >1400 entries

In [26]:
    (all := pandas.concat([
        (r := repos.set_index("url"))
        ["description stargazerCount forkCount licenseInfo tags".split()]
        .join(r.index.to_series().str.rpartition("/")[2].rename("name"))
        .rename(columns=dict(licenseInfo="license")),
        things.set_index("public_url")["name tags license description".split()],
        patterns.set_index(
            "https://www.ravelry.com/patterns/library/" + patterns.permalink
        )["name notes tags".split()].rename(columns=dict(notes="description"))
    ], axis=0))

,description,stargazerCount,forkCount,license,tags,name
https://github.com/ai-collection/ai-collection,The Generative AI Landscape - A Collection of ...,8019.0,796.0,MIT License,"[artificial intelligence, collections, ai, ass...",ai-collection
https://github.com/OptiKey/OptiKey,OptiKey - Full computer control and speech wit...,4325.0,505.0,GNU General Public License v3.0,"[eye tracking, eyetracking, eye tracker, eyes,...",OptiKey
https://github.com/brunopulis/awesome-a11y,A curate list about A11Y,1860.0,145.0,Creative Commons Zero v1.0 Universal,"[accessibility, wai aria, wcag, a11y, awesome ...",awesome-a11y
https://github.com/Stypox/dicio-android,Dicio assistant app for Android,933.0,87.0,GNU General Public License v3.0,"[assistant, assistive technology, personal ass...",dicio-android
https://github.com/cboard-org/cboard,Augmentative and Alternative Communication (AA...,674.0,188.0,GNU General Public License v3.0,"[aac, autism, cerebral palsy, progressive web ...",cboard
...,...,...,...,...,...,...
https://www.ravelry.com/patterns/library/unplanned-extubation-prevention-mitts,I designed this pattern according to the speci...,NaN,NaN,NaN,"[Mittens, preemie, icord, seamless, written pa...",Unplanned Extubation Prevention Mitts
https://www.ravelry.com/patterns/library/senior-socks-with-handles,These socks were designed for my elderly grand...,NaN,NaN,NaN,"[Tutorial, unisex, adult, toe up, written patt...",Senior socks with handles
https://www.ravelry.com/patterns/library/realistic-soft-packer--willy,"[Please note, this is not a physical product, ...",NaN,NaN,NaN,"[Mature Content, seamed, 3 dimensional, photot...",Realistic soft packer/ willy
https://www.ravelry.com/patterns/library/chill-buddy-ice-pack,This pattern is available for [free on my blog...,NaN,NaN,NaN,"[Medical, Other, Medical, Other, written patte...",Chill Buddy Ice Pack


In [27]:
    all.index.to_series().apply(urllib.parse.urlparse).apply(
        operator.attrgetter("netloc")
    ).value_counts().to_frame()

,count
www.ravelry.com,644
www.thingiverse.com,491
github.com,256


In [28]:
    counts = all.tags.explode().str.lower().value_counts()
    (tags := counts.head(100)).to_frame("tags").T.style.set_caption(
        "aggregated tags from all of the services"
    )

tags,written pattern,assistive technology,medical,in the round,one piece,accessibility,medical device accessory,assistivetech,screen reader,worked flat,other,adaptive,therapy aid,adult,seamless,unisex,disability,seamed,medical device access,phototutorial,assistive device,ribbed,occupational therapy,wai aria,wcag,bottom up,3 dimensional,female,stripes colorwork,textured,teen,chart,handicap,disabled,prosthetic,rectangle,rehabilitation,assistive,ergotherapie,hand,python3,child,chrome extension,top cuff down,video tutorial,firefox extension,mature,scalameta,scala,a11y,neovim plugin,opera extension,haskell,buttonholes,wheelchair,muff,braille,other accessibility,mid-calf,prosthetic hand,limited mobility,animal,"beanie, toque",top down,toe up,buttoned,aac,icord,prosthesis,lace,fitted,reversible,bobble or popcorn,aid,shawl / wrap,mobility aid accessory,amigurumi,assistive tech,discapacidad,kitchener,toddler,cables,therapy,male,arduino,occupational,technology,joystick,e-nable,blind,heel flap,switch access,communication,ceapat,imserso,nvda,preemie,noshaping,ipad,at switch
tags,541,446,392,252,235,230,226,226,224,201,190,190,166,163,157,116,114,112,99,99,92,85,79,77,76,75,71,67,58,56,54,46,45,45,45,44,41,41,39,38,36,35,34,33,33,33,32,32,32,32,32,32,32,32,31,31,30,30,29,28,28,27,26,26,25,25,24,23,23,23,22,21,21,21,20,20,20,19,19,19,19,19,19,18,18,18,18,18,17,17,17,17,17,17,17,17,16,16,16,15


In [29]:
all[~all.index.duplicated()].to_json("at.json.gz")